In [ ]:
import socket

def send_tcp_message(host, port, message):
    # Create a TCP/IP socket
    sock = socket.socket(socket.AF_INET, socket.SOCK_STREAM)

    try:
        # Connect the socket to the host and port
        sock.connect((host, port))

        # Send the message
        sock.sendall(message.encode())

    finally:
        # Close the socket
        sock.close()


In [ ]:
# asincio

In [ ]:
import asyncio, socket

async def handle_client(client):
    loop = asyncio.get_event_loop()
    request = None
    while request != 'quit':
        request = (await loop.sock_recv(client, 255)).decode('utf8')
        response = str(eval(request)) + '\n'
        await loop.sock_sendall(client, response.encode('utf8'))
    client.close()

async def run_server():
    server = socket.socket(socket.AF_INET, socket.SOCK_STREAM)
    server.bind(('localhost', 15555))
    server.listen(8)
    server.setblocking(False)

    loop = asyncio.get_event_loop()

    while True:
        client, _ = await loop.sock_accept(server)
        loop.create_task(handle_client(client))

asyncio.run(run_server())

In [ ]:
import asyncio, socket

async def handle_client(reader, writer):
    request = None
    while request != 'quit':
        request = (await reader.read(255)).decode('utf8')
        response = str(eval(request)) + '\n'
        writer.write(response.encode('utf8'))
        await writer.drain()
    writer.close()

async def run_server():
    server = await asyncio.start_server(handle_client, 'localhost', 15555)
    async with server:
        await server.serve_forever()

asyncio.run(run_server())

In [ ]:
import asyncio
from asyncio.streams import StreamReader, StreamWriter

async def handle_client(reader: StreamReader, writer: StreamWriter):
    """
    Handle communication with a client.

    Args:
        reader (StreamReader): The reader object for receiving data from the client.
        writer (StreamWriter): The writer object for sending data to the client.
    """
    client_address = writer.get_extra_info('peername')
    print(f'New connection from {client_address}')

    while True:
        data = await reader.read(1024)
        if not data:
            break

        message = data.decode('utf-8')
        print(f'Received message: {message}')

        # Process the message or perform any desired operations here

        # Send a response
        response = f'Received message "{message}"\n'
        writer.write(response.encode('utf-8'))
        await writer.drain()

    print(f'Connection from {client_address} closed')
    writer.close()

async def start_server():
    """
    Start the TCP server.

    Note:
        Modify the HOST and PORT variables to your desired values.
    """
    HOST = 'localhost'  # change this to your desired host
    PORT = 12345  # change this to your desired port

    server = await asyncio.start_server(
        handle_client, HOST, PORT)

    print(f'TCP server is listening on {HOST}:{PORT}...')

    async with server:
        await server.serve_forever()

# Run the server
asyncio.run(start_server())


In [ ]:
import sys
from importlib import reload
import socket
sys.path.append(
    r'C:\Users\stein\OneDrive\Documents\_Work\_code\ARPES-ASTRID\smartscan'
)
import smartscan.comms as scom

In [ ]:
reload(scom)

In [ ]:
def send_tcp_messge(host,port,msg):
    with socket.socket(socket.AF_INET, socket.SOCK_STREAM) as s:
        s.connect((host, port))
        s.sendall(msg.encode())
        data = s.recv(1024)
    return data

def recieve_tcp_message(host, port):
    """
    Recieves a message through the TCP socket and verifies its checksum.
    """
    with socket.socket(socket.AF_INET, socket.SOCK_STREAM) as s:
        s.bind((host, port))
        s.listen()
        conn, addr = s.accept()
        with conn:
            print('Connected by', addr)
            while True:
                data = conn.recv(1024)
                if not data:
                    break
                conn.sendall(data)

In [ ]:
send_tcp_messge('localhost', 12345, 'hi')

In [ ]:
send_tcp_messge('localhost', 12345, 'hi again')